In [11]:
from datasets import load_dataset
import cohere

In [12]:
cohere_api_key = ""
hf_api_key = ""
with open("../tokens/COHERE.txt", "r") as f:
    cohere_api_key = f.read().strip()
with open("../tokens/HF_TOKEN.txt", "r") as f:
    hf_api_key = f.read().strip()
co = cohere.Client(cohere_api_key)

In [13]:
data = load_dataset("LeoZotos/immu_full", split="train", token=hf_api_key )

In [14]:
def get_embeddings(batch):
    response = co.embed(
        texts=batch['Question_with_options'],
        model='embed-multilingual-v3.0',
        input_type='search_document' # Or 'search_query', depending on your use case
    )
    # The function must return a dictionary where the keys are the new column names
    return {"emb": response.embeddings}

data_with_embeddings = data.map(
    get_embeddings,
    batched=True,
    batch_size=96  # Cohere's API limit is 96 for this model. Adjust if needed.
)

# Now `data_with_embeddings` is a new Dataset object with the 'emb' column
print(data_with_embeddings)

Parameter 'function'=<function get_embeddings at 0x7f96dde1ed40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 843/843 [00:06<00:00, 124.50 examples/s]

Dataset({
    features: ['Question', 'Selectieveld', 'Figure', 'Chapter', 'Exam_and_Year', 'Answer_Key', 'Answer_A', 'Answer_B', 'Answer_C', 'Answer_D', 'Question_with_options', 'has_content_distractors', 'correct_answer_rate', 'distractor1_rate', 'distractor2_rate', 'distractor3_rate', 'RIT', 'emb'],
    num_rows: 843
})


In [20]:
print(data_with_embeddings[0]['emb'])

[-0.027557373, 0.012687683, 0.10601807, -0.015274048, -0.06311035, -0.018844604, -0.03062439, -0.014816284, -0.013809204, 0.044952393, -0.012069702, -0.060333252, 0.014907837, -0.04031372, 0.00819397, -0.016921997, 0.029968262, -0.038208008, 0.033294678, 0.02796936, 0.0014543533, 0.023345947, 0.007965088, -0.07836914, -0.03918457, 0.061065674, 0.036712646, -0.047790527, 0.06384277, 0.037475586, 0.005455017, -0.028335571, -0.024414062, -0.0032978058, 0.0046577454, 0.021118164, 0.027038574, -0.025405884, 0.0072784424, 0.024932861, 0.00012141466, -0.0050315857, -0.0055236816, 0.021316528, -0.007320404, 0.0030956268, 0.009017944, 0.002565384, 0.007534027, 0.034851074, 0.0013523102, 2.6285648e-05, -0.0008239746, -0.0068969727, 0.018569946, 0.004760742, -0.000118374825, -0.011779785, 0.047607422, 0.013206482, 0.021011353, 0.0067596436, 0.010536194, 0.017074585, -0.013046265, -7.241964e-05, -0.0041885376, 0.046325684, 0.007850647, 0.024108887, 0.018539429, 0.03451538, 0.054473877, -0.01423645

In [19]:
data_with_embeddings.push_to_hub(
    repo_id="LeoZotos/immu_full",
    commit_message="Add Cohere v3 embeddings to 'emb' column",
    token=hf_api_key,
    private=True
)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/LeoZotos/immu_full/commit/5cd1dc8005359af954a39b86411e84a36008c79a', commit_message="Add Cohere v3 embeddings to 'emb' column", commit_description='', oid='5cd1dc8005359af954a39b86411e84a36008c79a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/LeoZotos/immu_full', endpoint='https://huggingface.co', repo_type='dataset', repo_id='LeoZotos/immu_full'), pr_revision=None, pr_num=None)